# Animations In VCS<a id='top' class="tocSkip"> </a>

This Notebook demonstrates how to create animations in VCS

Animations are rendered via FFMPEG

The CDAT software was developed by LLNL. This tutorial was written by Charles Doutriaux. This work was performed under the auspices of the U.S. Department of Energy by Lawrence Livermore National Laboratory under Contract DE-AC52-07NA27344.

[Download the Jupyter notebook](Animations.ipynb)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparing-The-Notebook" data-toc-modified-id="Preparing-The-Notebook-1">Preparing The Notebook</a></span></li><li><span><a href="#Example-1:-Simple-2D-data-animation" data-toc-modified-id="Example-1:-Simple-2D-data-animation-2">Example 1: Simple 2D data animation</a></span><ul class="toc-item"><li><span><a href="#Prepare-data" data-toc-modified-id="Prepare-data-2.1">Prepare data</a></span></li><li><span><a href="#Create-Frames" data-toc-modified-id="Create-Frames-2.2">Create Frames</a></span></li><li><span><a href="#Create-Animation" data-toc-modified-id="Create-Animation-2.3">Create Animation</a></span></li></ul></li><li><span><a href="#Example-2:-1D-data-moving-behind-a-1D-target-data" data-toc-modified-id="Example-2:-1D-data-moving-behind-a-1D-target-data-3">Example 2: 1D data moving behind a 1D target data</a></span><ul class="toc-item"><li><span><a href="#Let's-prepare-some-data" data-toc-modified-id="Let's-prepare-some-data-3.1">Let's prepare some data</a></span></li><li><span><a href="#Prepare-graphic-methods-and-templates" data-toc-modified-id="Prepare-graphic-methods-and-templates-3.2">Prepare graphic methods and templates</a></span></li><li><span><a href="#Create-each-frame/png" data-toc-modified-id="Create-each-frame/png-3.3">Create each frame/png</a></span></li><li><span><a href="#Create-animation" data-toc-modified-id="Create-animation-3.4">Create animation</a></span></li><li><span><a href="#Slowing-Down" data-toc-modified-id="Slowing-Down-3.5">Slowing Down</a></span></li><li><span><a href="#Speeding-up-the-animation" data-toc-modified-id="Speeding-up-the-animation-3.6">Speeding up the animation</a></span></li></ul></li><li><span><a href="#Example-3:-Rotating-Globe" data-toc-modified-id="Example-3:-Rotating-Globe-4">Example 3: Rotating Globe</a></span><ul class="toc-item"><li><span><a href="#Preparing-the-data" data-toc-modified-id="Preparing-the-data-4.1">Preparing the data</a></span></li><li><span><a href="#Preparing-the-vcs-objects" data-toc-modified-id="Preparing-the-vcs-objects-4.2">Preparing the vcs objects</a></span></li><li><span><a href="#Preparing-the-images" data-toc-modified-id="Preparing-the-images-4.3">Preparing the images</a></span></li><li><span><a href="#Creating-the-animation" data-toc-modified-id="Creating-the-animation-4.4">Creating the animation</a></span></li><li><span><a href="#Controlling" data-toc-modified-id="Controlling-4.5">Controlling</a></span></li><li><span><a href="#Animating-Globe-and-Data" data-toc-modified-id="Animating-Globe-and-Data-4.6">Animating Globe and Data</a></span></li></ul></li></ul></div>

# Preparing The Notebook
[Back to top](#top)

We need some sample data

In [ ]:
import requests
r = requests.get("https://uvcdat.llnl.gov/cdat/sample_data/clt.nc",stream=True)
with open("clt.nc","wb") as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:  # filter local_filename keep-alive new chunks
            f.write(chunk)

# Example 1: Simple 2D data animation

In this example we will show a simple 2D animation

[Back to top](#top)


## Prepare data

[Back to Top](#top)

In [ ]:
import cdms2
f=cdms2.open("clt.nc")
clt = f("clt")

## Create Frames
[Back to top](#top)

In [ ]:
# make directory for pngs if not present
import os
if not os.path.exists("pngs"):
    os.makedirs("pngs")
    
import vcs
x=vcs.init(bg=True)

for i in range(clt.shape[0]):
    x.clear()
    x.plot(clt[i])
    x.png("pngs/2d_%s.png" % str(i).zfill(6))

## Create Animation
[Back to top](#top)

In [ ]:
import glob
name = "2d_animation.mp4"

# Create animation into file
x.ffmpeg(name,sorted(glob.glob("pngs/2d*png")))

# Example 2: 1D data moving behind a 1D target data

In this example we will show a fix 1D *target* dataset and a moving red dataset in the back

[Back to top](#top)


## Let's prepare some data
[Back to Top](#top)

In [ ]:
import cdutil
import numpy
# Open data file
f=cdms2.open("clt.nc")

# reads in data
s=f("clt")

# Computes time serie
ts=cdutil.averager(s,axis="xy")

# remove the mean
mean = ts.mean()
ts -= mean

# create some random data with some extra points to fake the move
Nextra = 120
ts2 = numpy.random.rand(len(ts)+Nextra)*4.-2.

# When plotting we will get data for 2 extra years than our target grid
# Prepare the "faxe time" axis
fake_time_axis = cdms2.createAxis(numpy.arange(-12,132))
fake_time_axis.designateTime()
fake_time_axis.id = "time"
fake_time_axis.units="months since 1979"


## Prepare graphic methods and templates
[Back to top](#top)

In [ ]:
# Create a "blank" template
blank = vcs.createtemplate()
blank.blank()
blank.data.priority=1 # turn only data area

# Create the "black" 1D line
black = vcs.create1d()
black.linecolor ="black"
black.markercolor = [0,0,0,0]
black.linewidth = 4
black.datawc_y1= -3
black.datawc_y2 = 3
black.datawc_x1 = -12
black.datawc_x2 = 132

# And from it let's create the "red" moving one
red = vcs.create1d(source=black.name)
red.linecolor="red"
red.linewidth=2
red.linetype = "dot"

## Create each frame/png
[Back to top](#top)

In [ ]:
x=vcs.init(bg=True)
for i in range(Nextra-24):
    # Get some subset of data
    tsub = cdms2.MV2.array(ts2[i:i+len(ts)+24]) # Two years worth of extra data
    # Aplly fake axis to make it look like it moves
    tsub.setAxis(0,fake_time_axis)
    # clear and plot
    x.clear()
    x.plot(ts,black)
    x.plot(tsub,red,blank)
    # save data to a png file
    x.png("pngs/1d_%s.png" % str(i).zfill(6))

## Create animation
[Back to top](#top)

In [ ]:
name = "1d_animation.mp4"

# Create animation into file
x.ffmpeg(name,sorted(glob.glob("pngs/1d*png")))

## Slowing Down
[Back to top](#top)

In [ ]:
import glob
name = "1d_animation_slow.mp4"

# Create animation into file
x.ffmpeg(name,sorted(glob.glob("pngs/1d*png")),rate=2)

## Speeding up the animation
[Back to top](#top)

In [ ]:
import glob
name = "1d_animation_fast.mp4"

# Create animation into file
x.ffmpeg(name,sorted(glob.glob("pngs/1d*png")),rate=200)

# Example 3: Rotating Globe

In this example we will display a rotating earth
[Back to top](#top)

## Preparing the data
[Back to top](#top)

In [ ]:
f=cdms2.open("clt.nc")
clt=f("clt",time=slice(0,1),longitude=(0,361),squeeze=1) # Read the time slice and squeeze it out

## Preparing the vcs objects
[Back to top](#top)

In [ ]:
# create new canvas
x = vcs.init(bg=True)

# Create projection
polar = vcs.createprojection("rotate")
polar.type = "orthographic"
polar.centerlongitude=0.
polar.centerlatitude=45.

# Create isofill method
iso = vcs.createisofill()
iso.levels = [a for a in range(0,110,10)]
iso.fillareacolors = vcs.getcolors(iso.levels)

iso.projection = polar

## Preparing the images
[Back to top](#top)

In [ ]:
for centerLongitude in range(0,366,5):
    polar.centerlongitude = centerLongitude
    x.clear()
    x.plot(clt,iso)
    x.png("pngs/rotate_%s.png" % str(centerLongitude).zfill(6))

## Creating the animation
[Back to top](#top)

In [ ]:
name = "rot_animation.mp4"

# Create animation into file
print("files used",glob.glob("pngs/rotate_*png"))
x.ffmpeg(name,sorted(glob.glob("pngs/rotate_*png")), rate=15)

## Controlling 
[Back to Top](#top)

But it is rotating **THE WRONG WAY!!!*** 

Why?

FFMPEG uses the images passed by the list in the order passed.

Look above the order is with **growing** centerLongitude

Let's fix this by passing the image sequence in **reversed** order

In [ ]:
name = "rot_animation_correct_order.mp4"

# Create animation into file
x.ffmpeg(name,sorted(glob.glob("pngs/rotate_*png"))[::-1], rate=15)

## Animating Globe and Data
[Back to Top](#top)

In [ ]:
# Create pngs
clt = f("clt",longitude=(0,361))
for i,centerLongitude in enumerate(range(0,361,4)):
    polar.centerlongitude = centerLongitude
    x.clear()
    x.plot(clt[i],iso)
    x.png("pngs/all_%s.png" % str(centerLongitude).zfill(6))

In [ ]:
name = "rot_animation_all.mp4"

# Create animation into file
x.ffmpeg(name,sorted(glob.glob("pngs/all_*png"))[::-1], rate=10)